In [1]:

import torch
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset, DataLoader
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from src.data.dataset import PatchFromH5Dataset, stratified_split, plot_class_distributions
from src.rl.train import ViTUCBTrainer
from transformers import (
    TrainingArguments,
    EarlyStoppingCallback,
)
from src.rl.modelling import ViT_UCB_Pruning
import evaluate
import os


/home/vcivale/UNI_UCB2/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["WANDB_PROJECT"] = "ViT-Pruning-Project"

In [3]:
IMG_SIZE = 224
TRAIN_BATCH_SIZE = 8
NUM_EPOCHS = 30

PRUNING_RATIO = 0.3

DEVICE = torch.device("cuda:1") if torch.cuda.is_available() else torch.device("cpu")

In [4]:
dataset = PatchFromH5Dataset(
    h5_dir='/equilibrium/datasets/TCGA-histological-data/hest/patches/patches/',
    transform=transforms.Compose([
        transforms.Resize(IMG_SIZE),
        transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),])
)

In [5]:
labels = dataset.labels

In [6]:
# Crea un DataFrame con indici e label
df = pd.DataFrame({
    "index": np.arange(len(labels)),
    "label": labels
})

# Trova il numero di elementi della classe minoritaria
min_count = df["label"].value_counts().min()

# Per ogni classe, seleziona min_count elementi a caso
undersampled_df = (
    df.groupby("label", group_keys=False)
      .apply(lambda x: x.sample(n=min_count, random_state=42)).reset_index(drop=True)
)

# Mischia gli indici
undersampled_indices = undersampled_df["index"].sample(frac=1, random_state=42).tolist()


/tmp/ipykernel_796240/3282357617.py:13: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=min_count, random_state=42)).reset_index(drop=True)


In [7]:
undersampled_labels = [labels[i] for i in undersampled_indices]

trainval_idx, test_idx = train_test_split(
    undersampled_indices,
    test_size=0.3,
    stratify=undersampled_labels,
    random_state=42
)

# Ottieni i label corrispondenti per il secondo split
trainval_labels = [labels[i] for i in trainval_idx]

# Split: train vs val
train_idx, val_idx = train_test_split(
    trainval_idx,
    test_size=0.3,
    stratify=trainval_labels,
    random_state=42
)

# Crea i subset
train_dataset = Subset(dataset, train_idx)
val_dataset   = Subset(dataset, val_idx)
test_dataset  = Subset(dataset, test_idx)

In [8]:
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

In [9]:
run_name = f"vit-ucb-pruning-{PRUNING_RATIO}"  

In [10]:
labels_num = len(np.unique(dataset.labels))

print(f"Number of classes: {labels_num}")
model = ViT_UCB_Pruning(model_name="hf-hub:MahmoodLab/uni", 
    pretrained=True, 
    n_classes=labels_num, 
    keep_ratio=PRUNING_RATIO,        
    exclude_cls=False
)

Number of classes: 27
Loading source model 'hf-hub:MahmoodLab/uni'...


09/03/2025 12:07:48 - INFO - timm.models._builder - Loading pretrained weights from Hugging Face hub (MahmoodLab/uni)


In [11]:
training_args = TrainingArguments(
    # --- Gestione dei salvataggi e output ---
    output_dir=f"/equilibrium/datasets/TCGA-histological-data/ViT-UCB-Pruning-checkpoints/Ratio-{PRUNING_RATIO}",  # Directory temporanea, non userà molto spazio
    report_to="wandb",                   
    push_to_hub=True,                    
    hub_model_id=f"Yuto2007/ViT-UCB-Pruning",
    run_name=run_name, 

    # --- Pianificazione del training ---
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    
    # --- Learning Rate Scheduler ---
    learning_rate=1e-1,
    warmup_steps=500,                     
    weight_decay=0.01,

    # --- Valutazione e Best Model ---
    eval_strategy="epoch",          
    save_strategy="epoch",               
    load_best_model_at_end=True,         
    metric_for_best_model="eval_loss",    
    greater_is_better=False,             

    logging_strategy="steps",
    logging_steps=500,
    save_total_limit=1,    

    fp16=False             
)

In [12]:
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=7,
    early_stopping_threshold=0.0,
)

In [13]:
def vit_ucb_collate_fn(batch):
    pixel_values = []
    labels = []

    for item in batch:
        patch = item['pixel_values']
        label = item['labels']

        # Assicuriamoci che la patch abbia sempre 3 dimensioni (C,H,W)
        if patch.ndim == 2:  # scala di grigi
            patch = patch.unsqueeze(0)
        elif patch.ndim == 3 and patch.shape[2] == 3:  # HWC -> CHW
            patch = patch.permute(2, 0, 1)

        pixel_values.append(patch)
        labels.append(label)

    # Stack sicuro
    pixel_values = torch.stack(pixel_values)  # (B,C,H,W)
    labels = torch.tensor(labels, dtype=torch.long)

    return {"pixel_values": pixel_values, "labels": labels}

In [14]:
trainer = ViTUCBTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,           
    compute_metrics=compute_metrics,      
    callbacks=[early_stopping_callback],
    data_collator=vit_ucb_collate_fn  
)


In [ ]:
print("🚀 Inizio dell'addestramento...")
trainer.train()

🚀 Inizio dell'addestramento...


wandb: Currently logged in as: vincenzo-civale (vincenzo-civale-universi-degli-studi-di-firenze) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


OutOfMemoryError: Caught OutOfMemoryError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/vcivale/UNI_UCB2/.venv/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 97, in _worker
    output = module(*input, **kwargs)
  File "/home/vcivale/UNI_UCB2/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1751, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/vcivale/UNI_UCB2/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1762, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/vcivale/UNI_UCB2/src/rl/modelling.py", line 183, in forward
    x, score_delta = block(x, counter=counter, ucb_enabled=ucb_enabled, ucb_count_score=self.ucb_count_scores[i])
  File "/home/vcivale/UNI_UCB2/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1751, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/vcivale/UNI_UCB2/.v

: 

In [ ]:
print("\n🚀 Valutazione finale sul test set con il modello migliore...")
test_results = trainer.predict(test_dataset)

In [ ]:
print("Risultati del Test Set:")
print(test_results.metrics)

In [ ]:
import wandb
wandb.log({"final_test_metrics": test_results.metrics})

In [ ]:
wandb.finish()
print("\n✅ Addestramento e valutazione completati.")